# Preparing `Moral-Stories` dataset for textual entailment
***
This notebook converts a subset of the moral-stories dataset to a sentence format that NLI models understand. The goal is to show, that the task of deciding whether an action is in accordance or violation of a norm is (almost) equivalent to the task of textual entailment.

# Loading the `Moral-Stories` dataset
***
The dataset and code can be found <a href="https://github.com/demelin/moral_stories">here</a>.\
The authors provide 12k unique norms and, for some reason, additional 700k variations of the same norms, just with NaN fields every now and then. Zero additional information, but maybe I am overlooking something here?
* Might be for different tasks? But then they only provide a single label which is always 1 for any NaN rows...

In [1]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
from ailignment.datasets import get_accuracy_metric, join_sentences, tokenize_and_split
import pandas as pd
import datasets
import transformers
import numpy as np
from ailignment import sequence_classification

pd.set_option('display.max_colwidth', 400)

#transformers.logging.set_verbosity_warning()

from collections import Counter

import spacy
from spacy import displacy
from tqdm import tqdm
import pandas as pd

In [2]:
dataframe = get_moral_stories()
nlp = spacy.load("en_core_web_sm")
# run everything through spacy for part of speech tags
dataframe["norm_parsed"] = dataframe["norm"].apply(nlp)

In [3]:
def get_pos_until_verb(doc):
    '''
    Returns a tuple of Part-of-speech tags up until the first verb
    Returns ("EMPTY") if no VERB is present
    '''
    l = [d.pos_ for d in doc]
    if "VERB" not in l: return ("EMPTY",)
    return tuple(l[:l.index("VERB")])

In [4]:
dataframe["norm_pos"] = dataframe["norm_parsed"].apply(lambda x: [d.pos_ for d in x])

In [5]:
dataframe["pos_verb"] = dataframe["norm_parsed"].apply(get_pos_until_verb)
#pos_verb = dataframe.groupby("pos_verb")

# Untying the *is* from the *ought*
***
We are not to come up with new norms. Rather, we like to get rid of the usually ambiguous normative judgements. Hence, we aim to split the moral value assigned to a norm from the situation it applies to. Checking for norm violation then reduces to other well-known problems such as question answering or textual entailment.

**TODO:** Surely there are smart names for the two categories of *situations that are judge-worthy* and their corresponding moral values.

We look for the most common ways that norms are phrased:
* "You should" or "You shouldn't"
* "It's" or "It is" followed by good, bad, etc.

Then, we want to translate them into "Did you ....?" and "Well, that's ..." pieces
* E.g. "You should back up so you can let people park." becomes
    * "Did you backup so you can let people park?"
    * "Yes" -> "Good"
    * "No" -> "Well, you should have!"

The norms only give value for either violation or adherence, but rarely both. We mirror the sentiment of the judgement by simple negation:
* "It is bad to hurt people"
    * "Did you hurt people?"
    * "Yes" --> "bad"
    * "No" --> "not bad"

Note, that we refrain from translating "not bad" to "good", since norms often aren't morally symmetric, i.e. not every norm has equally strong normativity for violation and adherence. E.g. positively committing a crime is punished much more explicitly than is not committing it. 

In [6]:
norms = dataframe["norm"].apply(str.lower)

f_should = lambda x: x.startswith("you should ") and not x.startswith("you should not")
shoulds = dataframe[norms.apply(f_should)]
print("\"You should\":", len(shoulds))

f_shouldnt = lambda x: x.startswith("you shouldn") or x.startswith("you should not")
shouldnts = dataframe[norms.apply(f_shouldnt)]
print("\"You shouldnt\":", len(shouldnts))

f_its = lambda x: x.startswith("it's") or x.startswith("it is")
its = dataframe[norms.apply(f_its)]
print("\"It's\"", len(its))

total = len(its)+len(shoulds)+len(shouldnts)
print("Covered", total ,"of",len(dataframe), f"({100*total/len(dataframe):.2f}%) norms")

"You should": 741
"You shouldnt": 2302
"It's" 8087
Covered 11130 of 11999 (92.76%) norms


### Untying `you should` sentences
***
First, find the most common grammatical structures:

In [7]:
common_pos = shoulds["norm_pos"].apply(lambda x: tuple(x[:3]))
common_pos = shoulds.groupby(common_pos)
most_common = common_pos["ID"].count().sort_values(ascending=False)
most_common

norm_pos
(PRON, AUX, VERB)    564
(PRON, AUX, ADV)     173
(PRON, AUX, AUX)       3
(PRON, AUX, PRON)      1
Name: ID, dtype: int64

Only two seem to be of relevance:
* "You should VERB"
    * You should help the ones in need
* "You should ADV"
    * You should always, You should never

While the first is straightforward to de-value, the latter at least requires more sophisticated reasoning capabilities concerning quantification and negation of cases.

For now, we simply strip the initial "You should" from the former, and the "You should ADV" from the latter to obtain the judgements.

In [8]:
# do first group of (PRON, AUX, VERB)
g0 = common_pos.get_group(("PRON","AUX","VERB")).copy()
g0["norm_devalued"] = g0["norm_parsed"].apply(lambda x: x[2:].text)
g0["value"] = "You should"

# second group of (PRON, AUX, ADV)
g1 = common_pos.get_group(("PRON","AUX","ADV")).copy()
g1["norm_devalued"] = g1["norm_parsed"].apply(lambda x: x[3:].text)
g1["value"] = g1["norm_parsed"].apply(lambda x: "You should " + x[2].text)

shoulds_devalued = pd.concat([g0,g1])

### Untying `you should not` sentences
***
First, find the most common grammatical structures:

In [9]:
common_pos = shouldnts["norm_pos"].apply(lambda x: tuple(x[:4]))
common_pos = shouldnts.groupby(common_pos)
most_common = common_pos["ID"].count().sort_values(ascending=False)
most_common

norm_pos
(PRON, AUX, PART, VERB)     2236
(PRON, AUX, PART, ADV)        35
(PRON, AUX, PART, AUX)        19
(PRON, AUX, PART, ADJ)         3
(PRON, AUX, PART, ADP)         3
(PRON, AUX, PART, NOUN)        3
(PRON, AUX, PART, PART)        1
(PRON, AUX, PART, PUNCT)       1
(PRON, AUX, VERB, NOUN)        1
Name: ID, dtype: int64

Only one seems to be of relevance:
* "You should not VERB"
    * You should not spit in someone's face

Similarly to the "You should" cases, we strip "You should not" as the value-judgement.

In [10]:
shouldnts_devalued = common_pos.get_group(("PRON","AUX","PART", "VERB")).copy()
shouldnts_devalued["norm_devalued"] = shouldnts_devalued["norm_parsed"].apply(lambda x: x[3:].text)
shouldnts_devalued["value"] = "You should not"

### Untying `it is` sentences
***
First, find the most common grammatical structures up until the first verb:

In [11]:
#common_pos = its["norm_pos"].apply(lambda x: tuple(x[:5]))
common_pos = its.groupby("pos_verb")
most_common = common_pos["ID"].count().sort_values(ascending=False)
most_common[:20]

pos_verb
(PRON, AUX, ADJ, PART)                    6518
(PRON, AUX, PART, ADJ, PART)               373
(PRON, AUX, ADJ, PART, PART)               344
(PRON, AUX)                                236
(PRON, AUX, ADJ, PART, ADV)                105
(EMPTY,)                                   101
(PRON, AUX, ADJ, PART, AUX)                 73
(PRON, AUX, ADJ, ADP, NOUN, PART)           49
(PRON, AUX, ADV, ADJ, PART)                 35
(PRON, AUX, PART, ADJ, PART, PART)          20
(PRON, AUX, ADJ, ADV, NOUN)                 16
(PRON, AUX, ADJ, ADP)                       14
(PRON, AUX, ADJ, ADP, DET, NOUN, PART)      13
(PRON, AUX, NOUN, PART)                     10
(PRON, AUX, ADJ, NOUN, PART)                 9
(PRON, AUX, PART)                            9
(PRON, AUX, PART, ADJ, PART, AUX)            8
(PRON, AUX, ADJ)                             6
(PRON, AUX, ADJ, ADP, PRON, PART)            5
(PRON, AUX, ADJ, CCONJ, ADJ, PART)           5
Name: ID, dtype: int64

The situation is a little more complex here, it seems:
1. "It is ADJ to", It's wrong to become addicted to gambling.
    * Value: ADJ, strip until first verb
2. "It is not ADJ to", It's not okay to invade someone else's privacy.
    * Like 1.
3. (PRON, AUX, ADJ, PART, PART). Two subgroups, which are both currently ignored
    1. "It is ADJ not to"
    2. "It is ADJ to not"

Similarly to the "You should" cases, we strip "You should not" as the value-judgement.

In [12]:
groups = []
# (PRON, AUX, ADJ, PART)
g = common_pos.get_group(("PRON","AUX","ADJ", "PART")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[4:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2].text)
groups.append(g)

In [13]:
# (PRON, AUX, PART, ADJ, PART)
g = common_pos.get_group(("PRON","AUX","PART","ADJ", "PART")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[5:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2:4].text)
groups.append(g)

In [14]:
# (PRON, AUX, ADJ, PART, PART)
g = common_pos.get_group(("PRON","AUX","ADJ","PART", "PART")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[5:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2:4].text)
# currently ignored

In [15]:
# (PRON, AUX)
g = common_pos.get_group(("PRON","AUX")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[5:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2:4].text)
# currently ignored

In [16]:
# finally, stitch everything together
its_devalued = pd.concat(groups)

# Imperatives and values
***
By splitting up value-judgements from the norms we end up with imperatives describing what the action looks like and the value one should expect to receive from it.

In [17]:
dataframe_devalued = pd.concat([shoulds_devalued, shouldnts_devalued, its_devalued])
total_split = len(dataframe_devalued)
print("After devaluation, we cover", total_split ,"of",len(dataframe), f"({100*total_split/len(dataframe):.2f}%) norms")

After devaluation, we cover 9864 of 11999 (82.21%) norms


In [18]:
dataframe_devalued[["moral_action","norm_devalued","value"]].sample(10)

,moral_action,norm_devalued,value
418,Joan decides to clean her house during her day off.,sleep all day,lazy
1864,Frances decides to be open-minded and get to know her new neighbors better.,judge others simply because they are strange.,You should not
7390,John explains his situation to his friends and still tries to do less crowded more low key activites with them.,spend time with your friends.,good
4008,Sandra ends her relationship with Jeff with the help of her best friend and begins looking for a better partner.,cheat on your romantic partner.,wrong
5380,Gillian thanks her classmates and offers to help them when they need it.,be happy when your classmates help you out.,You should
780,Phil bakes the pot brownies in a metal pan and the regular ones in a glass pan.,distinguish your pot brownies from regular brownies.,You should
845,Brad tells Jennifer that she doesn't have to ask him twice and jumps on her.,have an intimate relationship with your partner.,good
576,Jill goes through the store slowly and carefuly to look for good prices.,find bargains.,good
11793,John rushes out into the street and stands near the cat.,care for animals in danger.,great
9252,Joe escorts Bill home and then heads home himself.,let a drunk friend walk home alone.,You should not


# Vertical protoype
***
Before bothering with the norms that were left out during the above steps, I'd like to show a proof of concept. Therefore, the next steps are:
1. Derive a set of subjectified actions from the normative imperatives. I will do this with a table of conjugated verbs and handcrafted rules for the pronouns you, your and yourself. What about the names??
2. Run the textual entailment experiment between pairs of subjectified action and either moral or immoral action

## Subjectified actions prototype
***
1. Get a list of conjugated verbs
2. Get all norms whose imperative verb is contained in the list
3. Find the actor's name from the actions


In [19]:
# load a list of conjugated english verbs from https://github.com/monolithpl/verb.forms.dictionary
verbs = pd.read_csv("https://raw.githubusercontent.com/monolithpl/verb.forms.dictionary/master/csv/verbs-dictionaries.csv",
                   sep="\t", 
                    names=["present simple 1st","present simple 3rd","past simple","past participle","present participle"])
base_forms = set(verbs["present simple 1st"].to_list())
verb_map = {a:b for i,(a,b) in verbs[verbs.columns[:2]].iterrows()}

In [20]:
# re run pos tagging on the imperatives
dataframe_devalued["norm_devalued_pos"] = dataframe_devalued["norm_devalued"].apply(nlp)

In [21]:
# filter out all norms for which we do not have an entry in our verb list
norm_verbs = dataframe_devalued["norm_devalued_pos"].apply(lambda x: x[0].text)
dataframe_devalued = dataframe_devalued[norm_verbs.apply(lambda x: x in base_forms)]
print("After filtering out unknown verbs, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

After filtering out unknown verbs, we cover 9803 of 11999 (81.70%) norms


In [22]:
# apply pos tagging to moral actions to find sentences starting with a name
dataframe_devalued["moral_action_parsed"] = dataframe_devalued["moral_action"].apply(nlp)

In [23]:
# find the most common pos tags at the start of the actions
pos_groups = dataframe_devalued.groupby(dataframe_devalued["moral_action_parsed"].apply(lambda x: x[0].pos_))
counts = pos_groups["ID"].count().sort_values(ascending=False)
print(counts)

moral_action_parsed
PROPN    8910
NOUN      520
ADJ       124
ADV        64
PUNCT      57
ADP        44
AUX        34
VERB       15
DET        13
SCONJ      11
INTJ        6
PART        2
PRON        2
NUM         1
Name: ID, dtype: int64


In [24]:
# after visual examination, the first three categories seem to only contain names
# pos_groups["moral_action"].get_group(counts.index[2]).to_list()
dataframe_devalued = pd.concat([pos_groups.get_group(counts.index[i]) for i in [0,1,2]])
print("After focusing on known names, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

After focusing on known names, we cover 9554 of 11999 (79.62%) norms


In [25]:
# next up, gather actor names
dataframe_devalued["actor_name"] = dataframe_devalued["moral_action_parsed"].apply(lambda x: x[0].text)

### Switching persons
***
We need to also translate the pronouns in the imperatives from second (you, your) to third person (his/hers, him/her). Unfortunately, this requires the gender of the actor.

Here, we use package `gender-guesser`, as found here https://github.com/lead-ratings/gender-guesser

It assigns either andy, female, male, mostly female/male or unknown gender.

Once again, the ambiguity of the english language hits us:
* "you" might either stand for he/she or him/her
* Since only around 400 rows are affected, we ignore them for the protoype

In [26]:
import gender_guesser.detector as gg
det = gg.Detector()
dataframe_devalued["actor_gender"] = dataframe_devalued["actor_name"].apply(lambda x: det.get_gender(x,"usa"))
print(dataframe_devalued["actor_gender"].groupby(dataframe_devalued["actor_gender"]).count().sort_values(ascending=False))

actor_gender
male             5414
female           2196
mostly_male       868
andy              441
mostly_female     405
unknown           230
Name: actor_gender, dtype: int64


In [27]:
# filter on genders
dataframe_devalued["actor_gender"].replace("mostly_male","male", inplace=True)
dataframe_devalued["actor_gender"].replace("mostly_female","female", inplace=True)

dataframe_devalued = dataframe_devalued[dataframe_devalued["actor_gender"].apply(lambda x: "male" in x)]
print("After focusing on known genders, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

After focusing on known genders, we cover 8883 of 11999 (74.03%) norms


In [28]:
# get rid of all sentences with a "you", since we can't properly conjugate(?) it
dataframe_devalued = dataframe_devalued[dataframe_devalued["norm_devalued"].apply(lambda x: " you " not in x and not x.endswith(" you"))]
print("After filtering 'you's from norms, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

After filtering 'you's from norms, we cover 8445 of 11999 (70.38%) norms


In [29]:
# we check whether a norm story contains any 2nd person pronouns
male_pron_map = {"you":"he","your":"his","yours":"his","yourself":"himself"}
female_pron_map = {"you":"she","your":"her","yours":"hers","yourself":"herself"}

def translate_row(row):
    # actor name + 3rd person verb + rest of imperative
    # replace pronouns with 3rd person version
    verb = verb_map[row["norm_devalued_pos"][0].text]
    
    doc = row["norm_devalued_pos"]
    pron_map = male_pron_map if row["actor_gender"] == "male" else female_pron_map
    imperative = doc[:len(doc) if not doc[-1].is_punct else len(doc)-1]
    imperative = " ".join([pron_map.get(x.text, x.text) for x in imperative[1:]])

    story = " ".join([row["actor_name"], verb, imperative])
    return story


dataframe_devalued["norm_story"] = dataframe_devalued.apply(translate_row, axis=1)

## Incentive or prohibition?
***
In order to decide whether an action $A$ is moral or immoral, the worded value (e.g. manipulative, good, bad) needs to be interpretet as moral or immoral. Here we make the case, that sentiment analysis expresses whether a word was meant to be positive or negative. Consequently, we use the estimated sentiments as numerical normative values.

In [30]:
# Estimate sentiments using huggingface
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
value_map = {v:classifier(v)[0]["label"] for v in dataframe_devalued["value"].unique()}
dataframe_devalued["norm_sentiment"] = dataframe_devalued["value"].apply(value_map.get)

c:\users\kiehne\workspace\moral-stories-notebook\env\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [31]:
# save a stripped version of the data for later use
dataframe_devalued.to_pickle("../data/moral_stories_proto.dat")

In [32]:
# save a version of the data without all the spacy nlp data
data_light = dataframe_devalued[['ID', 'norm', 'situation', 'intention', 'moral_action', 'moral_consequence', 
                                 'immoral_action', 'immoral_consequence', 'norm_devalued', 'value', 'actor_name',
                                 'norm_sentiment', 'actor_gender', 'norm_story']]
data_light.to_pickle("../data/moral_stories_proto_light.dat")

***
# END OF NOTEBOOK

In [33]:
data_light["norm"]

,ID,norm,situation,intention,moral_action,moral_consequence,immoral_action,immoral_consequence,norm_devalued,value,actor_name,norm_sentiment,actor_gender,norm_story
4162,3HSYG7LRBKWDT5F3HJ1O9VYLIINKK0,It's good to be a responsible person.,It's a hot day and John needs groceries and his two year old is in the car with him.,John wants to grocery shop quickly.,John takes his two year old with him to the store and shops quickly.,His child does okay and John is still able to shop effectively.,John leaves his two year old in the hot car and rushes to get groceries from the store alone.,His child could get very sick and die and John could get arrested.,be a responsible person.,good,John,POSITIVE,male,John is a responsible person


In [ ]:
from transformers import Trainer, TrainingArguments
import torch

dataframe = dataframe_devalued[dataframe_devalued.columns[:8]]
test_split = 0.2
batch_size = 12
model = "distilbert-base-uncased"
#model = "albert-base-v2"
action_dataframe = make_action_classification_dataframe(dataframe)
input_columns = ["norm", "action"]
action_dataframe["task_input"] = join_sentences(action_dataframe, input_columns)
dataset = datasets.Dataset.from_pandas(action_dataframe)
dataset = dataset.train_test_split(test_size=test_split)

def data_all(tokenizer):
    return tokenize_and_split(dataset, tokenizer, "task_input")
def data_small(tokenizer):
    train, test = data_all(tokenizer)
    train = train.shuffle(seed=42).select(range(1000))
    test = test.shuffle(seed=42).select(range(1000))
    return train, test

training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    #weight_decay=wd,
    #learning_rate=lr,
    logging_dir='./logs',
    logging_steps=500,
    save_steps=1000000,
    save_total_limit=0,
    evaluation_strategy="epoch",
)

r = sequence_classification(data_all, model, training_args, get_accuracy_metric())
acc = [x["eval_accuracy"] for x in r if "eval_accuracy" in x]
print(acc)

In [ ]:
[[x["eval_accuracy"] for x in r if "eval_accuracy" in x] for r in results]

# WIP: Get score output from LM
***
Question: Is there a better way to sample from generated LM outputs?

In [ ]:
from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelWithLMHead.from_pretrained(model)

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=False, top_p=0.95, top_k=60,
                        return_dict_in_generate=True, output_attentions=False,
                        output_hidden_states=True, output_scores=True)
#generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

p = torch.softmax(outputs.scores[0], dim=1)

print(p.max())

# WIP: Data augmentation with NER
***
Idea: Use Named entity recognition to find and replace persons etc. 

In [ ]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
from ailignment import join_sentences, tokenize_and_split, get_accuracy_metric
dataframe = get_moral_stories()
columns = dataframe.columns[1:]
print("Running NER on columns", columns.to_list())

In [ ]:
texts = join_sentences(dataframe ,columns, "\n")

In [ ]:
import spacy
from spacy import displacy
from tqdm import tqdm
import pandas as pd

nlp = spacy.load("en_core_web_sm")

ner_pipe = nlp.pipe(tqdm(texts), disable=['tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
docs = [x for x in ner_pipe]

displacy.render(docs[0], style="ent")

In [ ]:
from collections import Counter

def get_frequent_entity(doc, entity="PERSON", n=1):
    '''
    Returns the highest number of occurences of an
    entity in the NER doc.
    '''
    occurences = [(x.text, x.label_) for x in doc.ents if x.label_ == entity]
    c = Counter(occurences)
    ents = []
    for item, count in c.most_common(n):
        ents.append([x for x in doc.ents if (x.text, x.label_) == item])
    
    if n == 1 and len(ents) != 0:
        ents = ents[0]
    return ents

In [ ]:
persons = [get_frequent_entity(x, "PERSON",n=1) for x in docs]
# we are interested in the simplest case, where the NER found
# exactly 6 matches
matches = [x for x in persons if len(x) == 6]
print(f"Found {len(matches)} matches")

In [ ]:
m = matches[0]
displacy.render(m[0].doc, "ent")

In [ ]:
def replace_entity(ents, s):
    '''
    Replaces all occurences of entities in `ents` with `s`.
    `ents` is a list of entities as returned by `doc.ents`
    from an NER pipeline, they need to be from the same doc!
    '''
    offset = 0
    text = ents[0].doc.text
    new_text = ""
    for ent in ents:
        start = ent.start_char
        end = ent.end_char
        left = text[offset:start]
        new_text += left + s
        offset = end
    new_text += text[offset:]
    return new_text

In [ ]:
n_docs = [replace_entity(m, "Niklas").split("\n") for m in matches]

In [ ]:
n_docs = [m[0].doc.text.split("\n") for m in matches]

In [ ]:
dataframe_replaced = pd.DataFrame(n_docs)
dataframe_replaced.columns = columns
dataframe_replaced.head()

In [ ]:
import datasets
test_split = 0.2
batch_size = 8

action_dataframe = make_action_classification_dataframe(dataframe_replaced)

input_columns = ["action"]
action_dataframe["task_input"] = join_sentences(action_dataframe, input_columns, " ")
dataset = datasets.Dataset.from_pandas(action_dataframe)
dataset = dataset.train_test_split(test_size=test_split)


In [ ]:

from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model)

train_data, test_data = tokenize_and_split(dataset, tokenizer, "task_input")

# for prototyping, optional
small_train_data = train_data.shuffle(seed=42).select(range(1000))
small_test_data = test_data.shuffle(seed=42).select(range(1000))

training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,                # how often to log
    save_steps=1000,
    save_total_limit=0,
    evaluation_strategy="epoch",     # when to run evaluation
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=small_train_data,   # training dataset
    eval_dataset=small_test_data,     # evaluation dataset
    compute_metrics=get_accuracy_metric,     # code to run accuracy metric
)
trainer.train()

In [1]:
from gender_guesser.detector import Detector

In [2]:
d = Detector()

In [ ]:
from ailignment.datasets.moral_stories import _lemmatize, get_moral_stories

In [ ]:
import spacy
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'textcat'])
STOP_WORDS = stopwords.words('english')
stories = get_moral_stories()
columns = ["moral_action", "immoral_action"]


In [ ]:
series = stories[columns[0]]

In [ ]:
def lemmatize_series(series, nlp, STOP_WORDS=None):
    '''
    Given a series of strings, returns a DataFrame(["lemmas", "tokens", "maps"])
    of the lemmatized strings according to `_lemmatize` function.
    '''
    # get rid of whitespace
    translation_table = str.maketrans(' ', ' ', punctuation)
    series = series.map(lambda x: x.translate(translation_table))
    series = series.map(lambda x: _lemmatize(x, nlp, STOP_WORDS))
    data = pd.DataFrame(series.to_list(), columns=["lemmas", "tokens", "maps"])
    return data

In [ ]:
y = series.map(lambda x: _lemmatize(x, nlp, STOP_WORDS))

In [ ]:
data = pd.DataFrame(y.to_list(), columns=["lemmas", "tokens", "maps"])


In [ ]:
data

In [ ]:
b

In [ ]:
c